In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.activations import sigmoid
from tensorflow.keras.activations import relu

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Dense
from tensorflow.keras.layers import Activation, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

from timeit import default_timer as timer

from sklearn import preprocessing
# Resets 
tf.compat.v1.reset_default_graph()
path = '../datasets/'

In [26]:
def AndreKeras(X_train,y_train,X_test,y_test,steps,batch,neurons,act,use_earlyStopFunction):

    model = Sequential()
    model.add(Dense(100,activation=act , input_shape=(X_train.shape[1],)) )
    model.add(Dense(neurons,activation=act))
    model.add(Dense(1))


    epochs = steps
    
    learning_rate = 0.0001
    opt = Adam(learning_rate = learning_rate)
    model.compile(loss="mean_squared_error", optimizer=opt)

    batch_size = batch
    
    if(use_earlyStopFunction):
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 1, epochs = steps,callbacks = [callback],verbose = False)
    else:
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 1, epochs = steps,verbose = False)
        
    return model,history    

def computeSquareDeviation(predictions, truth):
  squareDeviation = np.square(predictions-truth)
  return squareDeviation

In [22]:
#NYC DATASET

dataset = "nyc_taxi.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])


raw["hour"] = [t.hour for t in pd.DatetimeIndex(raw.timestamp)]
raw["minute"] = [t.minute for t in pd.DatetimeIndex(raw.timestamp)]
raw["day"] = [t.dayofweek for t in pd.DatetimeIndex(raw.timestamp)]
raw["month"] = [t.month for t in pd.DatetimeIndex(raw.timestamp)]
raw['year'] = [t.year for t in pd.DatetimeIndex(raw.timestamp)]
raw.drop(['timestamp'],inplace=True,axis=1)

data = raw.to_numpy()

X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = True

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

In [23]:
dataset = "Google_Stock_Price_Train.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])

raw["day"] = [t.day for t in pd.DatetimeIndex(raw.Date)]
raw["month"] = [t.month for t in pd.DatetimeIndex(raw.Date)]
raw['year'] = [t.year for t in pd.DatetimeIndex(raw.Date)]
raw.drop(['Date',"day","month"],inplace=True,axis=1)

raw['Volume'] = raw['Volume'].str.replace(',', '')
raw['Close'] = raw['Close'].str.replace(',', '')

data = raw.to_numpy(dtype='float')

X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = False

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))



relu 25
Time 327.1535539000006
NRMSE 0.02419597331328203


relu 50
Time 351.16153719999966
NRMSE 0.020679875648122157


relu 100
Time 345.9948610000001
NRMSE 0.02529713798277923


sigmoid 25
Time 344.5754401000004
NRMSE 0.02577481964237529


sigmoid 50
Time 355.64550919999965
NRMSE 0.021857556947521107


sigmoid 100
Time 365.36542980000013
NRMSE 0.031717386538884576


In [30]:
dataset = "Abalone.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])
map = {'M':0,'F':1,'I':2}
raw = raw.replace({'sex':map})

raw = raw.reindex(columns=['rings','length','diameter','height','weight.w','weight.s','weight.v','weight.sh','sex'])
display(raw.head())

data = raw.to_numpy(dtype='float')

X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = True

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

,rings,length,diameter,height,weight.w,weight.s,weight.v,weight.sh,sex
0,7,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,0
1,9,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1
2,10,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,0
3,7,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,2
4,8,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,2


In [ ]:
#BIKE DATASET
dataset = "train_bikeDemand.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])
raw.drop(raw.columns[0], axis=1,inplace=True)
cols = list(raw.columns)
a, b = cols.index('holiday'), cols.index('count')
cols[b], cols[a] = cols[a], cols[b]
raw = raw[cols]

data = raw.to_numpy(dtype='float')



X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = True

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

In [31]:
#REC CENTER DATASET
dataset = "rec-center-hourly.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1,2])

raw["day"] = [t.day for t in pd.DatetimeIndex(raw.timestamp)]
raw["month"] = [t.month for t in pd.DatetimeIndex(raw.timestamp)]
raw.drop(["timestamp"],inplace=True,axis=1)
display(raw.head())

data = raw.to_numpy(dtype='float')

X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = True

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

,kw_energy_consumption,day,month
0,21.2,2,7
1,16.4,2,7
2,4.7,2,7
3,4.7,2,7
4,4.6,2,7




relu 25
Time 19.32884860000013
NRMSE 6.975757677857954


relu 50
Time 12.050458899999285
NRMSE 7.517476632550677


relu 100
Time 15.129576200000884
NRMSE 8.465183858235623


sigmoid 25
Time 42.062565500000346
NRMSE 7.85478600919484


sigmoid 50
Time 28.912588600000163
NRMSE 8.452301120121426


sigmoid 100
Time 29.253272699999798
NRMSE 8.41170098252783


In [27]:
#COVID DATASET
dataset = "covid_chile.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])
raw["day"] = [t.day for t in pd.DatetimeIndex(raw.date)]
raw["month"] = [t.month for t in pd.DatetimeIndex(raw.date)]
raw['year'] = [t.year for t in pd.DatetimeIndex(raw.date)]
raw.drop(["date"],inplace=True,axis=1)
raw = raw.reindex(columns=['confirmed','day','month','year','deaths','recovered'])
display(raw.head())


data = raw.to_numpy(dtype='float')

X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = False

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

,confirmed,day,month,year,deaths,recovered
0,0,23,1,2020,0,0
1,0,24,1,2020,0,0
2,0,25,1,2020,0,0
3,0,26,1,2020,0,0
4,0,27,1,2020,0,0




relu 25
Time 49.473234400000365
NRMSE 0.00958055511724152


relu 50
Time 49.12855010000021
NRMSE 0.010322786873624503


relu 100
Time 50.8849252
NRMSE 0.010501529710550636


sigmoid 25
Time 49.34387989999959
NRMSE 0.037716444029695886


sigmoid 50
Time 48.82452039999953
NRMSE 0.05611927117248238


sigmoid 100
Time 49.037908099999186
NRMSE 0.07506050194481481


In [34]:
#AVOCADOS

avocados = pd.read_csv(path + 'filtered_avocados.csv')
new_columns = pd.DataFrame(avocados["Date"].str.split('-').tolist(), columns = ['yr', 'month', 'day'])

# Change data type of new columns from str to int32
new_columns["day"] = new_columns["day"].astype(np.int32)
new_columns["month"] = new_columns["month"].astype(np.int32)
# Attach new columns
avocados["day"] = new_columns["day"]
avocados["month"] = new_columns["month"]
avocados["type"]=pd.Categorical(avocados["type"])
avocados["region"]=pd.Categorical(avocados["region"])
df_type = pd.get_dummies(avocados['type'], prefix = 'category')
df_region = pd.get_dummies(avocados['region'], prefix = 'category')

# Drop Date, type and region
del avocados["Date"]
del avocados["type"]
del avocados["region"]

# Attach the one-hot encodings columns to other integer ones
avocados_cleaned=pd.concat([avocados, df_type, df_region], axis=1, sort=False)

# Shuffle data using the pandas function sample, with fraction as 1 since we want all data 
avocados = avocados_cleaned.sample(frac=1)
# Converto to numpy array
avocados = avocados.to_numpy()

X = avocados[:,1:]
y = avocados[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = True

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

In [28]:
dataset = "auto-mpg.csv"
raw = pd.read_csv(path + dataset, header=0, skiprows=[1])
raw.drop(["car name"],inplace=True,axis=1)
display(raw.head())

data = raw.to_numpy(dtype='float')
X = data[:,1:]
y = data[:,:1]

means = np.mean(y,axis=0)
stds = np.std(y,axis=0)
y = (y-means)/stds

std_Ys = stds[0]
mean_Ys = means[0]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123,shuffle = True)


batch = 1
steps = int(len(X_train) / batch)

use_earlyStopFunction = False

for f in ["relu","sigmoid"]:
    for n_ in [25,50,100]:
        start = timer()
        model_ ,history_ = AndreKeras(X_train,y_train,X_test,y_test,steps,batch,n_,f,use_earlyStopFunction)
        end = timer()
        t_ = end - start
        test_predicted = model_.predict(X_test)
        predictions = (test_predicted * std_Ys) + mean_Ys
        target = (y_test * std_Ys) + mean_Ys
        squareDeviation = computeSquareDeviation(predictions, target)
        print("\n")
        print(f,n_)
        print("Time",t_)
        print("NRMSE",np.sqrt(np.nanmean(squareDeviation)) / np.nanstd(predictions))

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,15.0,8,350.0,165,3693,11.5,70,1
1,18.0,8,318.0,150,3436,11.0,70,1
2,16.0,8,304.0,150,3433,12.0,70,1
3,17.0,8,302.0,140,3449,10.5,70,1
4,15.0,8,429.0,198,4341,10.0,70,1




relu 25
Time 38.126587999999174
NRMSE 0.40084193740754476


relu 50
Time 39.37913870000011
NRMSE 0.4392796104390117


relu 100
Time 39.87838269999884
NRMSE 0.4365824832264944


sigmoid 25
Time 39.05691210000077
NRMSE 0.3713614534410912


sigmoid 50
Time 39.83130980000169
NRMSE 0.37819787885129647


sigmoid 100
Time 41.003256600000896
NRMSE 0.36774535782274065
